In [ ]:
from __future__ import print_function

import datetime
import numpy as np
import glob
import warnings

from multiprocessing import Pool

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs
    
from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status



#### Helpful resource:
http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

In [ ]:
data_dir = "../saved_runs/cluster_parameter_study/"

In [ ]:
data_dir = "../saved_runs/cluster_parameter_study/"
id = "f8644401"
run_summary = RunSummary(data_dir=data_dir, id=id)

In [ ]:
s = Simulation_Status.from_run_summary(run_summary)

In [ ]:
s.status

In [ ]:
def parse_into_simulations(data_dir_and_id):
    # wrapper function to allow multiprocessing + map
    data_dir, id = data_dir_and_id
    run_summary = RunSummary(data_dir=data_dir, id=id)

    simulation = Simulation.from_run_summary(run_summary)
    
    inputs = Inputs(os.path.join(data_dir, id + "_inputs.dat"))
    simulation_inputs = Simulation_Inputs.from_Inputs(id, inputs)
    
    simulation_status = Simulation_Status.from_run_summary(run_summary)
    
    
    return simulation, simulation_inputs, simulation_status
        

# Check which rows already exist

In [ ]:
for sim in session.query(Simulation):
    print(sim)

In [ ]:
for sim_inputs in session.query(Simulation_Inputs):
    print(sim_inputs)

# Add files to database in batch

In [ ]:
def add_entire_dir_to_database(data_dir):
    overview_filenames = glob.glob(os.path.join(data_dir, "*overview.dat"))
    ids = [None]*len(overview_filenames)
    for k, filename in enumerate(overview_filenames):
        ids[k] = os.path.basename(filename).split("_overview.dat")[0]

    ids = ids[:4]

    mappable_inputs = [(data_dir, id) for id in ids]
    
    pool = Pool(4)
    simulation_tuples = pool.map(parse_into_simulations, mappable_inputs)
    for simulation, simulation_inputs, simulation_status in simulation_tuples:
        simulation.add_or_update_to_table()
        simulation_inputs.add_or_update_to_table()
        simulation_status.add_or_update_to_table()
add_entire_dir_to_database(data_dir)

# Check statuses from database

In [ ]:
statuses = np.array([status.status for status in session.query(Simulation_Status)])
counts = {status : 0 for status in Simulation_Status.possible_statuses}

for status in statuses:
    counts[status] += 1

print(counts)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Error"):
    print(simulation_status.id)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Running"):
    print(simulation_status.id)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Unknown"):
    print(simulation_status.id)

# Restart if ready

In [ ]:
for status in session.query(Simulation_Status):
    if status.status == "Ready":
#         status.create_new_batch_file_and_switch_to_running()
        pass

# Shut down SQL session when done

In [ ]:
session.commit()

In [ ]:
session.close()